In [ ]:
# default_exp core

# ohawf

> 

In [ ]:
# hide
from nbdev.showdoc import *

# Requirements
    pip install google-auth-oauthlib
    pip install google-api-python-client

In [ ]:
# export
"""
Google OAuth2 login module.
"""
import json
import pickle
from urllib.request import urlopen
import google.auth.transport.requests
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow


class OhAwf:
    """Login to Google services."""

    def __init__(self):
        self.url = "https://www.googleapis.com/oauth2/v1/tokeninfo?access_token="
        self.scopes = ["https://www.googleapis.com/auth/webmasters.readonly"]
        self.creds = None
        with open("credentials.json", "rb") as handle:
            self.creds = json.load(handle)
        self.run()

    def login(self):
        """Start web-based Google OAuth2 login prompt."""
        flow = InstalledAppFlow.from_client_config(self.creds, self.scopes)
        self.creds = flow.run_console()
        with open("credentials.pkl", "wb") as handle:
            pickle.dump(self.creds, handle)
        return self.creds

    def refresh_token(self):
        """Refresh token to make new logins generally not required."""
        with open("credentials.pkl", "rb") as handle:
            self.creds = pickle.load(handle)
        cred_url = self.url + self.creds.token
        try:
            cred_response = urlopen(cred_url)
        except:
            request = google.auth.transport.requests.Request()
            self.creds.refresh(request)
            with open("credentials.pkl", "wb") as handle:
                pickle.dump(self.creds, handle)
        return self.creds
    
    def run(self):
        try:
            self.creds = self.refresh_token()
        except:
            self.creds = self.login()
        return self.creds

if __name__ == "__main__":
    owauf = OhAwf()
    credentials = owauf.run()
    print(credentials)